<img width="400" src="https://nomeroff.net.ua/images/logo/nomeroff_net.svg" alt="Nomeroff Net. Automatic numberplate recognition system" align="left"/>

## Quick recognition example.

In [1]:
# Import all necessary libraries.
import os
import sys
import matplotlib.image as mpimg
import warnings
warnings.filterwarnings('ignore')


# change this property
NOMEROFF_NET_DIR = os.path.abspath('../')

# specify the path to Mask_RCNN if you placed it outside Nomeroff-net project
MASK_RCNN_DIR = os.path.join(NOMEROFF_NET_DIR, 'Mask_RCNN')

MASK_RCNN_LOG_DIR = "../logs/"
MASK_RCNN_MODEL_PATH = "../models/mask_rcnn_numberplate_0700.h5"
OPTIONS_MODEL_PATH =  "../models/numberplate_options_64x128-1_2019_2_15.h5"
OCR_NP_UKR_TEXT =  "../models/anpr_ocr_ua_1_2_11-cpu.h5"
OCR_NP_EU_TEXT =  "../models/anpr_ocr_eu_2-cpu.h5"

sys.path.append(NOMEROFF_NET_DIR)

In [2]:
# Import license plate recognition tools.
from NomeroffNet import  filters, RectDetector, TextDetector,  OptionsDetector, Detector, textPostprocessing

# Initialize npdetector with default configuration file.
nnet = Detector(MASK_RCNN_DIR, MASK_RCNN_LOG_DIR)

# Load weights in keras format.
nnet.loadModel(MASK_RCNN_MODEL_PATH)

# Initialize rect detector with default configuration file.
rectDetector = RectDetector()

# Initialize text detector.
ukrTextDetector = TextDetector("ocr_ukr")
ukrTextDetector.load(OCR_NP_UKR_TEXT)
euTextDetector = TextDetector("ocr_eu")
euTextDetector.load(OCR_NP_EU_TEXT)

# Initialize train detector.
optionsDetector = OptionsDetector()
optionsDetector.load(OPTIONS_MODEL_PATH)

Using TensorFlow backend.


Instructions for updating:
Create a `tf.sparse.SparseTensor` and use `tf.sparse.to_dense` instead.


In [3]:
# Detect numberplate
img_path = 'images/example1.jpeg'
img = mpimg.imread(img_path)
NP = nnet.detect([img])

In [4]:
# Generate image mask.
cv_img_masks = filters.cv_img_mask(NP)
for img_mask in cv_img_masks:
    # Detect points.
    points = rectDetector.detect(img_mask)
    # Split on zones
    zone = rectDetector.get_cv_zonesBGR(img, points)
    
    # find standart
    regionId, stateId = optionsDetector.predict(zone)
    regionName = optionsDetector.getRegionLabel(regionId)

    # find text with postprocessing by standart
    if regionId in (1, 2):
        text = ukrTextDetector.predict(zone)
    else:
        text = euTextDetector.predict(zone)
        
    text = textPostprocessing(text, regionName)
    print(text)
    

AC4921CB
